In [79]:
import pandas as pd
import numpy as np
from functools import reduce

# Load data
pop = pd.read_csv("data/PopulationEstimates.csv", encoding='latin-1', dtype={'fipstxt': str}).rename(columns=lambda x: x.strip().lower().replace(' ', '_'))
poverty = pd.read_csv("data/Poverty2023.csv", encoding='latin-1').rename(columns=lambda x: x.strip().lower().replace(' ', '_'))
edu = pd.read_csv("data/Education2023.csv", encoding='latin-1').rename(columns=lambda x: x.strip().lower().replace(' ', '_'))
unemp = pd.read_csv("data/Unemployment2023.csv", encoding='latin-1').rename(columns=lambda x: x.strip().lower().replace(' ', '_'))
crime_rate = pd.read_csv('data/crime_rate.csv', encoding='latin-1')
temperature = pd.read_csv('data/temperature.csv', encoding='latin-1', comment='#')
precipitation = pd.read_csv('data/precipitation.csv', encoding='latin-1', comment='#')
type(pop['fipstxt'])
pop['fipstxt'] = pop['fipstxt'].apply(lambda x: f"{int(x):05d}" if pd.notnull(x) else np.nan)
pop['fipstxt'] = pop['fipstxt'].str.zfill(5)

# First subset by the attribute we want, then select the state and area name and create a new variable to store values in
pop = pop[pop['attribute'] == 'POP_ESTIMATE_2023'][['state', 'area_name', 'value','fipstxt']].rename(columns={'value': 'Population_Estimate','fipstxt': 'FP'})
poverty = poverty[poverty['attribute'] == 'PCTPOVALL_2023'][['stabr', 'area_name', 'value']].rename(columns={'stabr': 'state', 'value': 'Poverty_Percent'})
edu = edu[edu['attribute'] == "Bachelor's degree or higher, 2019-23"][['state', 'area_name', 'value']].rename(columns={'value': 'Bachelor_Or_Higher'})

precipitation['state'] = precipitation['ID'].str[:2]
precipitation_cleaned = precipitation[['state', 'Value', 'Name']].rename(columns={'Value': 'Avg_Precipitation', 'Name': 'area_name'})

temperature['state'] = temperature['ID'].str[:2]
temperature_cleaned = temperature[['state', 'Value', 'Name']].rename(columns={'Value': 'Avg_Temp', 'Name': 'area_name'})

crime_cleaned = crime_rate.copy()
crime_cleaned[['area_name', 'state']] = crime_cleaned['county_name'].str.split(", ", expand=True)
crime_cleaned['area_name'] = crime_cleaned['area_name'].str.upper()
crime_cleaned['state'] = crime_cleaned['state'].str.upper()
crime_cleaned['Crime_Rate_Per_100000'] = crime_cleaned['crime_rate_per_100000'].round(2)
crime_cleaned = crime_cleaned[['area_name', 'state', 'Crime_Rate_Per_100000']]

# Removing the , XX at the end of area names in unemp
unemp['area_name'] = unemp['area_name'].str.replace(",\\s*\\w{2}$", "", regex=True)
medianincome = unemp[unemp['attribute'] == 'Median_Household_Income_2022'][['state', 'area_name', 'value']].rename(columns={'value': 'Median_Income'})
unemp = unemp[unemp['attribute'] == 'Unemployment_rate_2023'][['state', 'area_name', 'value']].rename(columns={'value': 'Unemployment_Rate'})

# Combines datasets into a list to be used via full join
data_list = [pop, poverty, edu, unemp, medianincome, temperature_cleaned, precipitation_cleaned]

# Combines the data, normalizes the capitalization, and collapses any duplicate entries after basic data cleaning, and finally removes rows with missing values
combined_data = reduce(lambda left, right: pd.merge(left, right, on=['area_name', 'state'], how='outer'), data_list)
combined_data['area_name'] = combined_data['area_name'].str.upper()
combined_data['state'] = combined_data['state'].str.upper()
combined_data = combined_data.merge(crime_cleaned, on=['area_name', 'state'], how='outer')

combined_data['area_name'] = combined_data['area_name'].str.replace(r"/.*", "", regex=True).str.strip()
combined_data = combined_data.groupby(['area_name', 'state']).agg(lambda x: x.dropna().iloc[0] if not x.dropna().empty else np.nan).reset_index()
combined_data = combined_data.dropna()

print(combined_data.head())
# Save the combined data to a CSV file
combined_data.to_csv("CombinedData.csv", index=False, encoding='utf-8-sig')

          area_name state  Population_Estimate     FP  Poverty_Percent  \
0  ABBEVILLE COUNTY    SC              24434.0  45001             15.3   
1     ACADIA PARISH    LA              56489.0  22001             25.0   
2   ACCOMACK COUNTY    VA              33239.0  51001             17.3   
3        ADA COUNTY    ID             524673.0  16001              8.0   
4      ADAIR COUNTY    IA               7389.0  19001              9.8   

   Bachelor_Or_Higher  Unemployment_Rate  Median_Income  Avg_Temp  \
0              3332.0                3.7        50325.0      64.4   
1              5071.0                3.7        42981.0      70.3   
2              5314.0                3.5        56357.0      59.1   
3            153355.0                2.7        87748.0      52.8   
4              1052.0                2.4        66997.0      50.6   

   Avg_Precipitation  Crime_Rate_Per_100000  
0              37.73                 511.86  
1              62.29                 163.98  
2 

In [80]:
import geopandas as gpd
# Load the appropriate layer
gdf = gpd.read_file("data/Natl_WI.gdb", layer="NationalWalkabilityIndex")  # Replace with actual layer name
full_data = gdf.drop(columns=["geometry"])  # drop geometry if you just want tabular data
# We will use STATEFP and COUNTYFP as a "key" to merge with our combined dataset.
# There are 220,000 observations for 3,000 counties. 
# We need to average the walkability index (NatWalkInd) among all the blocks in each county.
# Weighted by area (Shape_Area) of the block.
df = full_data[["STATEFP","COUNTYFP","NatWalkInd","Shape_Area"]].copy()
# Combine STATEFP and COUNTYFP to FP.
df["FP"] = df["STATEFP"].astype(str).str.zfill(2) + df["COUNTYFP"].astype(str).str.zfill(3)
df = df.drop(columns=["STATEFP","COUNTYFP"])
import numpy as np
# Weighted average of NatWalkInd by FP
walkability_df = (
    df.groupby("FP")[["NatWalkInd", "Shape_Area"]]
      .apply(lambda g: (g["NatWalkInd"] * g["Shape_Area"]).sum() / g["Shape_Area"].sum())
      .reset_index(name="Walkability")
)
print(walkability_df.head())

      FP  Walkability
0  01001     3.267533
1  01003     4.104647
2  01005     3.703383
3  01007     4.165608
4  01009     4.360420


In [81]:
merged_df = pd.merge(combined_data, walkability_df, on='FP', how='inner')
print(len(merged_df))
print(merged_df.head())

3086
          area_name state  Population_Estimate     FP  Poverty_Percent  \
0  ABBEVILLE COUNTY    SC              24434.0  45001             15.3   
1     ACADIA PARISH    LA              56489.0  22001             25.0   
2   ACCOMACK COUNTY    VA              33239.0  51001             17.3   
3        ADA COUNTY    ID             524673.0  16001              8.0   
4      ADAIR COUNTY    IA               7389.0  19001              9.8   

   Bachelor_Or_Higher  Unemployment_Rate  Median_Income  Avg_Temp  \
0              3332.0                3.7        50325.0      64.4   
1              5071.0                3.7        42981.0      70.3   
2              5314.0                3.5        56357.0      59.1   
3            153355.0                2.7        87748.0      52.8   
4              1052.0                2.4        66997.0      50.6   

   Avg_Precipitation  Crime_Rate_Per_100000  Walkability  
0              37.73                 511.86     4.077864  
1              62

In [82]:
area = pd.read_csv("data/Land_Area.csv", encoding='utf-8-sig')
area.rename(columns={"STCOU":"FP", "LND010190D":"Land_Area"}, inplace=True)
area["FP"] = area["FP"].astype(str).str.zfill(5)
merged_df2 = pd.merge(combined_data, area, on='FP', how='inner')
# There was actually one row with a land area of 0. I have edited in the Census value into the CSV beforehand
print(merged_df2.iloc[[277]])

len(merged_df2)




             area_name state  Population_Estimate     FP  Poverty_Percent  \
277  BROOMFIELD COUNTY    CO              76860.0  08014              5.1   

     Bachelor_Or_Higher  Unemployment_Rate  Median_Income  Avg_Temp  \
277             31270.0                3.0       114746.0      52.7   

     Avg_Precipitation  Crime_Rate_Per_100000  Land_Area  
277              10.21                  50.44      33.58  


3086

In [83]:
df = merged_df2.copy()
col="Land_Area"
indices = df[(df[col] == 0) | (df[col].isna())].index
index_list = indices.tolist()
print(f"length is {len(index_list)}")
print(index_list)

print(len(df))
print(len(merged_df2))

length is 0
[]
3086
3086
